In [3]:
import numpy as np

import scipy as sp
from scipy import spatial

import matplotlib.pyplot as plt # side-stepping mpl backend

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import pairwise
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
import heapq
import string
import re

from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter

with open("data/train.dat.txt", "r") as fh:
    #with open("TestData/Test/training_out.dat", "r") as fh:
    trainingData = fh.readlines()
print(len(trainingData))

with open("data/test.dat.txt", "r") as fh:
#with open("TestData/Test/test_out.dat", "r") as fh:
    testData = fh.readlines()
print(len(testData))


25000
25000
